In [1]:
import os

In [2]:
file_list = os.listdir('new-SASA')
name_list = [i.rsplit('.',1)[0] for i in file_list]

In [3]:
name_list[5]

'1zoy-C'

In [4]:
lenth_same_list = []
length_not_same_list = []
for i in name_list:
    with open('new-SASA/' + i + '.sasa') as f:
        sasa_contents = f.readlines()
        f.close()

    with open('new-fasta/' + i + '.fasta') as g:
        fasta_contents = g.read()
        g.close()

    if len(sasa_contents) == len(fasta_contents) :
        lenth_same_list.append(i)
    else:
        length_not_same_list.append(i)

In [5]:
len(lenth_same_list)

10282

In [7]:
from tqdm import tqdm
from itertools import groupby
import numpy as np

label_dic = {}
for i in tqdm(lenth_same_list):
    chian_name = i.split('-')[1]
    #print(chian_name)
    
    with open('IFP-renamed/' + i + '.IFP') as f:
        IFP_contents = f.readlines()[0][:-1]
        f.close()
    
    with open('new-SASA/'+i+'.sasa') as g:
        sasa_list = g.readlines()
        g.close()
    
    IFP_list = IFP_contents.split('|')
    #print(IFP_list)
    IFP_resn_list = []
    for j in IFP_list:
        if j[0:1] == chian_name:
            if 'HOH' not in j:
                ss = [''.join(list(g)) for k, g in groupby(j, key=lambda x: x.isdigit())]
                IFP_resn_list.append(ss[-1])
    
    #print(IFP_resn_list)
    result_list = []
    for m in sasa_list:
        add1 = False
        for n in IFP_resn_list:
            if m.split()[0] == n:
                add1 = True
                break
            else:
                pass
        if add1 == True:
            result_list.append('1')
        else:
            result_list.append('0')
    
    result_str = ''.join(result_list)
    #print(result_str)
    #break
    with open('new-IFP/' + i + '.IFP', 'w') as g:
        g.write(result_str)
        g.close()
    
    label_dic[i] = np.array(result_list).astype(np.float64)

100%|██████████| 10282/10282 [09:18<00:00, 18.42it/s]


In [8]:
np.save('part2-label',label_dic)

In [9]:
import numpy as np
np.save('part2-index',lenth_same_list)

In [11]:
import numpy as np
from sklearn.preprocessing import minmax_scale
from tqdm import tqdm

sasa_dic = {}
for i in tqdm(lenth_same_list):
    with open('new-SASA/' + i + '.sasa', 'r') as f:
        contents = f.readlines()
        f.close()
    
    sasa_contents = [j.split(' ')[1][:-1] for j in contents]
    tmp = np.array(sasa_contents)
    tmp = minmax_scale(tmp)
    float64_tmp = tmp.astype(np.float64)
    float64_tmp_T = float64_tmp.T
    sasa_dic[i] = float64_tmp.T


100%|██████████| 10282/10282 [00:58<00:00, 174.90it/s]


In [15]:
pssm_dic = {}
for i in tqdm(lenth_same_list):
    with open('new-pssm/' + i + '.pssm', 'r') as f:
        contents = f.readlines()
        f.close()

    pssm_contents = [j.split()[22:42] for j in contents[3:-6]]
    array_test = np.array(pssm_contents)
    array_test = array_test.astype(np.float64)
    array_test = array_test/100
    array_test = array_test.astype(np.float64)
    # print(array_test.shape)
    pssm_dic[i] = array_test


100%|██████████| 10282/10282 [05:45<00:00, 29.72it/s]


In [16]:
np.save('part2-pssm', pssm_dic)

In [8]:
import numpy as np
from sklearn.preprocessing import minmax_scale
from tqdm import tqdm

In [19]:
SS_dic = {}
for i in tqdm(lenth_same_list):
    with open('new-ss/%s.ss' %i) as f:
        contents = f.readlines()
        f.close()

    SS_contents = [j.split()[-3:] for j in contents[1:]]
    SS_array = np.array(SS_contents)
    SS_array = SS_array.astype(np.float64)
    SS_dic[i] = SS_array

 49%|████▊     | 4991/10282 [03:10<01:40, 52.45it/s]  

In [ ]:
np.save('part2-SS', SS_dic)